# Unstructured data analysis with LlamaExtract and JSONalyze Query Engine

In this notebook, we extract structured data from resume PDFs and run analytical queries on them with `LlamaExtract` and `JSONalyze Query Engine`. 

The steps are:
1. Infer schema from resume PDFs with `LlamaExtract`
2. Extract structured data (i.e. JSONs) from resume PDFs with `LlamaExtract`
3. Answer analytical queries with `JSONalyze Query Engine`

Additional Resources: 
- `LlamaExtract`: https://docs.cloud.llamaindex.ai/
- `JSONalyze Query Engine`: https://docs.llamaindex.ai/en/stable/examples/query_engine/JSONalyze_query_engine/

## Setup

Install `llama-extract` client library.

In [ ]:
%pip install llama-extract > /dev/null


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Follow [instruction](https://docs.cloud.llamaindex.ai/llamacloud/getting_started/api_key)  to get an API key from https://cloud.llamaindex.ai/

In [ ]:
import os

os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."

## Load data

For this demo, We use 3 sample resumes from [Resume Dataset](https://www.kaggle.com/datasets/gauravduttakiit/resume-dataset) from Kaggle (data is included in this repo).

In [ ]:
DATA_DIR = "data/resumes"

In [ ]:
fnames = os.listdir(DATA_DIR)
fnames = [fname for fname in fnames if fname.endswith(".pdf")]
fpaths = [os.path.join(DATA_DIR, fname) for fname in fnames]
fpaths

['data/resumes/14224370.pdf',
 'data/resumes/12780508.pdf',
 'data/resumes/19545827.pdf']

### Infer schema

First, let's infer a JSON schema that represent the inherent structure of our documents with `LlamaExtract`.

In [ ]:
from llama_extract import LlamaExtract

extractor = LlamaExtract()

In [ ]:
fpaths_for_infer_schema = fpaths[:1]
fpaths_for_infer_schema

['data/resumes/14224370.pdf']

In [ ]:
extraction_schema = await extractor.ainfer_schema(
    "Resume Schema 07_24", fpaths_for_infer_schema
)
extraction_schema.id

'94e09ba1-ed0d-4427-889e-014afb57dbeb'

Let's preview the inferred schema.

In [ ]:
from IPython.display import JSON

JSON(extraction_schema.data_schema)

<IPython.core.display.JSON object>

In [ ]:
extraction_schema.data_schema

{'type': 'object',
 'properties': {'Accountant': {'type': 'object',
   'properties': {'skills': {'type': 'string'},
    'summary': {'type': 'object',
     'properties': {'location': {'type': 'string'},
      'yearsOfExperience': {'type': 'integer'}}},
    'education': {'type': 'object',
     'properties': {'degree': {'type': 'string'},
      'honors': {'type': 'string'},
      'institution': {'type': 'string'},
      'fieldOfStudy': {'type': 'string'},
      'graduationYear': {'type': 'integer'}}},
    'experience': {'type': 'string'},
    'highlights': {'type': 'object',
     'properties': {'quickLearner': {'type': 'integer'},
      'teamOriented': {'type': 'integer'},
      'detailOriented': {'type': 'integer'},
      'deadlinesFocused': {'type': 'integer'},
      'goodAnalyticalSkills': {'type': 'integer'},
      'strongManagementSkills': {'type': 'integer'},
      'abilityToManageMultiTasks': {'type': 'integer'},
      'excellentCommunicationSkills': {'type': 'integer'}}},
    'tec

### (Optional) Update schema

Let's modify the schema to remove the `skills` field.

In [ ]:
new_schema = extraction_schema.data_schema.copy()

del new_schema["properties"]["Accountant"]["properties"]["skills"]

In [ ]:
from IPython.display import JSON

JSON(new_schema)

<IPython.core.display.JSON object>

In [ ]:
update_response = await extractor.aupdate_schema(extraction_schema.id, new_schema)

In [ ]:
print("Updated at", update_response.updated_at)

Updated at 2024-07-25 17:31:31.695033+00:00


In [ ]:
update_response.data_schema

{'type': 'object',
 'properties': {'Accountant': {'type': 'object',
   'properties': {'summary': {'type': 'object',
     'properties': {'location': {'type': 'string'},
      'yearsOfExperience': {'type': 'integer'}}},
    'education': {'type': 'object',
     'properties': {'degree': {'type': 'string'},
      'honors': {'type': 'string'},
      'institution': {'type': 'string'},
      'fieldOfStudy': {'type': 'string'},
      'graduationYear': {'type': 'integer'}}},
    'experience': {'type': 'string'},
    'highlights': {'type': 'object',
     'properties': {'quickLearner': {'type': 'integer'},
      'teamOriented': {'type': 'integer'},
      'detailOriented': {'type': 'integer'},
      'deadlinesFocused': {'type': 'integer'},
      'goodAnalyticalSkills': {'type': 'integer'},
      'strongManagementSkills': {'type': 'integer'},
      'abilityToManageMultiTasks': {'type': 'integer'},
      'excellentCommunicationSkills': {'type': 'integer'}}},
    'technicalSkills': {'type': 'object',


### Run extraction

Now that we have the schema, we can extract structured representation of our resume files.

In [ ]:
extraction_results = await extractor.aextract(extraction_schema.id, fpaths)



Extracting files:   0%|                                        | 0/3 [00:00<?, ?it/s]

Extracting files:  33%|██████████▋                     | 1/3 [00:08<00:16,  8.12s/it]

Extracting files:  67%|█████████████████████▎          | 2/3 [00:10<00:04,  4.61s/it]

Extracting files: 100%|████████████████████████████████| 3/3 [01:35<00:00, 31.97s/it]


Let's inspect extracted education data for each resume.

In [ ]:
for result in extraction_results:
    education = result.data["Accountant"]["education"]
    print(json.dumps(education, indent=2))

{
  "degree": "Bachelor's degree",
  "honors": "Cum Laude - Graduating With Honors",
  "institution": "University of Houston",
  "fieldOfStudy": "accounting",
  "graduationYear": 2005
}
{
  "degree": "Bachelor of Science",
  "honors": null,
  "institution": "University of Maryland",
  "fieldOfStudy": "Finance",
  "graduationYear": 2010
}
{
  "degree": "Bachelor of Arts",
  "honors": "Maintained a 4.0 GPA while completing 12 hours of upper level accounting courses.",
  "institution": "Marshall University",
  "fieldOfStudy": "Economics",
  "graduationYear": null
}


In [ ]:
resume_jsons = [x.data["Accountant"] for x in extraction_results]

### Run analytical queries

In [ ]:
%pip install jsonpath_ng > /dev/null
%pip install sqlite-utils > /dev/null


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

from llama_index.core.query_engine import JSONalyzeQueryEngine
from llama_index.llms.openai import OpenAI

os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
llm = OpenAI(model="gpt-4")
query_engine = JSONalyzeQueryEngine(list_of_dict=resume_jsons, llm=llm)

In [ ]:
response = query_engine.query("Tell me about candidates from University of Houston")

In [ ]:
response.response

"The candidate from University of Houston has a Bachelor's degree in accounting and graduated Cum Laude in 2005. They have 10 years of experience, with roles including Accountant, Tax Accountant, and Accounting Manager. They have worked in various locations, including Houston. They have strong technical skills, including proficiency in Adobe Reader, Drake, Ultra Tax, TurboTax, Microsoft Office, XP Windows Professional, Windows 7, Windows 8, PeopleSoft, and QuickBooks. They are a quick learner, team-oriented, detail-oriented, focused on deadlines, have good analytical and strong management skills, and have the ability to manage multiple tasks. They also have excellent communication skills."

In [ ]:
response.metadata["sql_query"]

"SELECT * FROM items WHERE education LIKE '%University of Houston%';"